In [1]:
from models.ji_two_node import two_nodes_optimized

import pandas as pd
from pythermalcomfort.utilities import body_surface_area
from pythermalcomfort.psychrometrics import p_sat_torr

# Import dataset
df_dataset = pd.read_csv('./dataset/230322_OlderPredictTc_data_thermal.csv')

In [2]:
def simulate_trial(sample: pd.DataFrame, clo_males: float, clo_females: float, met: float, Va: float, isCooling: bool) -> list:
    ids = sample.id_all.unique()
    all_core_temps = []
    all_skin_temps = []
    for i in ids:
        individual = pd.DataFrame(sample[sample.id_all == i])
        # Get parameters
        isFemale = individual.female.iloc[0]
        sex = 'female' if isFemale else 'male'
        clo = clo_females if isFemale else clo_males
        height = individual.height.iloc[0] / 100
        weight = individual.mass.iloc[0]

        # Calculate
        body_area = body_surface_area(weight, height)
        Ta = individual.ta_set.iloc[0]  # Operative temperature [oC]
        RH = individual.rh_set.iloc[0]  # Relative humidity [%]
        vapor_pressure = RH * p_sat_torr(Ta) / 100

        # Simulate cool environment
        cool_ta = 23
        cool_rh = 50
        cool_pressure = cool_rh * p_sat_torr(cool_ta) / 100
        initial_cooling = two_nodes_optimized(
            tdb=cool_ta,
            tr=cool_ta,
            v=Va,
            met=met,
            clo=clo,
            vapor_pressure=cool_pressure,
            wme=0,
            body_surface_area=body_area,
            p_atmospheric=101325,
            body_position="sitting",
            body_weight=weight,
            acclimatized=True,
            length_time_simulation=60
        )

        # Predict temperature
        if isCooling:
            # Heat
            results = two_nodes_optimized(
                tdb=Ta,
                tr=Ta,
                v=Va,
                met=met,
                clo=clo,
                vapor_pressure=vapor_pressure,
                wme=0,
                body_surface_area=body_area,
                p_atmospheric=101325,
                body_position="sitting",
                body_weight=weight,
                acclimatized=True,
                length_time_simulation = 225,
                initial_skin_temp = initial_cooling['skin_temp_hist'][-1],
                initial_core_temp = initial_cooling['core_temp_hist'][-1],
            )
            # Cooling
            new_results = two_nodes_optimized(
                tdb=cool_ta,
                tr=cool_ta,
                v=Va,
                met=met,
                clo=clo,
                vapor_pressure=cool_pressure,
                wme=0,
                body_surface_area=body_area,
                p_atmospheric=101325,
                body_position="sitting",
                body_weight=weight,
                acclimatized=True,
                length_time_simulation = 120,
                initial_skin_temp = results['skin_temp_hist'][-1],
                initial_core_temp = results['core_temp_hist'][-1],
            )
            results['skin_temp_hist'] += new_results['skin_temp_hist']
            results['core_temp_hist'] += new_results['core_temp_hist']
            # Back to heat
            new_results = two_nodes_optimized(
                tdb=Ta,
                tr=Ta,
                v=Va,
                met=met,
                clo=clo,
                vapor_pressure=vapor_pressure,
                wme=0,
                body_surface_area=body_area,
                p_atmospheric=101325,
                body_position="sitting",
                body_weight=weight,
                acclimatized=True,
                length_time_simulation = individual.shape[0] - 345,
                initial_skin_temp = results['skin_temp_hist'][-1],
                initial_core_temp = results['core_temp_hist'][-1],
            )
            results['skin_temp_hist'] += new_results['skin_temp_hist']
            results['core_temp_hist'] += new_results['core_temp_hist']
        else:
            results = two_nodes_optimized(
                tdb=Ta,
                tr=Ta,
                v=Va,
                met=met,
                clo=clo,
                vapor_pressure=vapor_pressure,
                wme=0,
                body_surface_area=body_area,
                p_atmospheric=101325,
                body_position="sitting",
                body_weight=weight,
                acclimatized=True,
                length_time_simulation=individual.shape[0],
                initial_skin_temp=initial_cooling['skin_temp_hist'][-1],
                initial_core_temp=initial_cooling['core_temp_hist'][-1],
            )

        # Extract data
        core_temp = results['core_temp_hist']
        skin_temp = results['skin_temp_hist']
        all_core_temps.append(core_temp)
        all_skin_temps.append(skin_temp)

    # Flatten lists before returning
    all_core_temps = [temp for sublist in all_core_temps for temp in sublist]
    all_skin_temps = [temp for sublist in all_skin_temps for temp in sublist]
    return all_core_temps, all_skin_temps

In [3]:
from helpers import get_sample

def run_and_save_trial(study, condition, clo_males, clo_females, met, Va, isCooling):
    sample = get_sample(study, condition)
    results = simulate_trial(sample, clo_males, clo_females, met, Va, isCooling)
    all_core_temps = results[0]
    all_skin_temps = results[1]
    print(len(all_core_temps))
    print(len(all_skin_temps))
    print(sample.shape[0])
    df = pd.DataFrame(all_core_temps, columns=["tre_predicted"])
    df["mtsk_predicted"] = all_skin_temps
    df.to_csv('results/ji-{}-{}.csv'.format(study, condition), index=False)

In [4]:
run_and_save_trial('heatwave 1 (prolonged)', 'hot', clo_males=0.15, clo_females=0.23, met=1.3, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'cool', clo_males=0.23, clo_females=0.23, met=1.3, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'temp', clo_males=0.23, clo_females=0.23, met=1.3, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'warm', clo_males=0.23, clo_females=0.23, met=1.3, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'hot', clo_males=0.23, clo_females=0.23, met=1.3, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 3 (cooling)', 'hot', clo_males=0.15, clo_females=0.23, met=1.3, Va=0.1, isCooling=True)

31860
31860
31860
7680
7680
7680
7680
7680
7680
17760
17760
17760
16800
16800
16800
10260
10260
10260
